In [1]:
# Setup Environment
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import psutil
import time
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from mpl_toolkits.mplot3d import Axes3D
from sklearn.svm import SVR
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import LabelEncoder
import torch
import seaborn as sns
import pickle
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, GATConv, GATv2Conv
from torch.nn import Linear
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import concurrent.futures
from concurrent.futures import ProcessPoolExecutor
import time
from data_processors.pull_and_process_data import master_function

if torch.cuda.is_available():
    print("CUDA GPU is available.")
    device = torch.device('cuda')
else:
    print("CUDA GPU is not available. Using CPU instead.")
    device = torch.device('cpu')
    
print(f"Current GPU device: {torch.cuda.get_device_name(torch.cuda.current_device())}")
print(f"Total RAM: {(psutil.virtual_memory().total / (1024**3)):.2f} GB")
print(f"Available RAM: {(psutil.virtual_memory().available / (1024**3)):.2f} GB")

CUDA GPU is available.
Current GPU device: NVIDIA A100-PCIE-40GB
Total RAM: 1006.92 GB
Available RAM: 963.08 GB


## Train model on mouse 1, save best model 

In [2]:
import wandb
user = wandb.login(relogin=True)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /nas/longleaf/home/cvita1/.netrc


In [2]:
from auto_hyperparameter_tuner import *
from torch.cuda.amp import GradScaler, autocast

param_space = {
    "wandb_project": "Predicting Visual Stimulus",
    "wandb_api_key": "7c8d251196fd96d2a93bfb6ffd0005ac030ce42b",
    "num_epochs": 50,
    "lr": 0.000815819,
    "temporal_hidden_dim": 760,
    "spatial_hidden_dim": tune.randint(15,20),
    "edge_threshold":.25,
    "early_stop_patience": 3,
    "early_stop_delta": 0.01,
    "batch_size": 16,
    "graph_batch_size": 16,
    "temporal_layer_dimension":1,
    "spatial_out_features": 1,
    "mouse_number": 715093703,
    "timesteps": 10,
    "Architecture": 'Static_STGAT',
    "num_samples": 1,
    "accumulation_steps": 4,  
    "graph_lr": .05, 
    "use_auto_corr_matrix": True,
    "file_path":"/proj/STOR/pipiras/Neuropixel/output/spike_trains_with_stimulus_session_715093703_10.pkl"
    }

trainer = ModelTrainer(param_space)
trainer.execute_tuning()

(_trainable pid=4026795) Loaded spike trains dataset: <class 'pandas.core.frame.DataFrame'>
(_trainable pid=4026795) X_val shape: torch.Size([476, 10, 2065, 1])
(_trainable pid=4026795) y_val shape: torch.Size([476, 1])
(_trainable pid=4026795)  
(_trainable pid=4026795) X (array): Matrix of number of batch_size, time_steps_per_frame, num_nodes, and number of features per node.
(_trainable pid=4026795) X.shape = (B, T, N, F)
(_trainable pid=4026795) X_train shape: torch.Size([4284, 10, 2065, 1])
(_trainable pid=4026795) X_test shape: torch.Size([1190, 10, 2065, 1])
(_trainable pid=4026795) X_train type: <class 'torch.Tensor'>
(_trainable pid=4026795)  
(_trainable pid=4026795) X_val shape: torch.Size([476, 10, 2065, 1])
(_trainable pid=4026795) y_val shape: torch.Size([476, 1])
(_trainable pid=4026795)  
(_trainable pid=4026795) y_shape = [batch_size, unique_frames_shown_per_10_timesteps]
(_trainable pid=4026795) y_train shape: torch.Size([4284, 1])
(_trainable pid=4026795) y_test shap

(_WandbLoggingActor pid=4026989) wandb: Currently logged in as: rayscarpenter (neuropixel-unc). Use `wandb login --relogin` to force relogin
(_WandbLoggingActor pid=4026989) wandb: wandb version 0.16.6 is available!  To upgrade, please run:
(_WandbLoggingActor pid=4026989) wandb:  $ pip install wandb --upgrade
(_WandbLoggingActor pid=4026989) wandb: Tracking run with wandb version 0.16.2
(_WandbLoggingActor pid=4026989) wandb: Run data is saved locally in /nas/longleaf/home/cvita1/ray_results/exp/_trainable_bdfac277_1_Architecture=Static_STGAT,accumulation_steps=4,batch_size=16,early_stop_delta=0.0100,early_stop_patience=3,e_2024-04-13_05-55-52/wandb/run-20240413_055606-bdfac277
(_WandbLoggingActor pid=4026989) wandb: Run `wandb offline` to turn off syncing.
(_WandbLoggingActor pid=4026989) wandb: Syncing run _trainable_bdfac277
(_WandbLoggingActor pid=4026989) wandb: ⭐️ View project at https://wandb.ai/neuropixel-unc/Predicting%20Visual%20Stimulus
(_WandbLoggingActor pid=4026989) wand

(_trainable pid=4026795) Test accuracy 2.86% is not higher than the existing highest accuracy 85.21%. Model not saved.
(_trainable pid=4026795) Epoch 1, Duration: 60.25s, Loss: 1.22, Train Acc: 0.96%, Test Acc: 2.86%
(_trainable pid=4026795) Total connections: 4264225
(_trainable pid=4026795) Edge sliver weights: tensor([0.7311, 0.5010, 0.5075, 0.5019, 0.5028], device='cuda:0',
(_trainable pid=4026795)        grad_fn=<SliceBackward0>)


Test Epoch 2/50:   0%|          | 0/75 [00:00<?, ?it/s]


(_trainable pid=4026795) Test accuracy 11.93% is not higher than the existing highest accuracy 85.21%. Model not saved.
(_trainable pid=4026795) Epoch 2, Duration: 60.16s, Loss: 1.13, Train Acc: 5.18%, Test Acc: 11.93%
(_trainable pid=4026795) Total connections: 4264225
(_trainable pid=4026795) Edge sliver weights: tensor([0.7311, 0.5023, 0.5127, 0.5069, 0.5037], device='cuda:0',
(_trainable pid=4026795)        grad_fn=<SliceBackward0>)


...

(_trainable pid=4026795) Test accuracy 67.06% is not higher than the existing highest accuracy 85.21%. Model not saved.
(_trainable pid=4026795) Epoch 4, Duration: 60.44s, Loss: 0.46, Train Acc: 65.45%, Test Acc: 67.06%
(_trainable pid=4026795) Total connections: 4264225
(_trainable pid=4026795) Edge sliver weights: tensor([0.7311, 0.5311, 0.5179, 0.4899, 0.5443], device='cuda:0',
(_trainable pid=4026795)        grad_fn=<SliceBackward0>)


Test Epoch 5/50:   0%|          | 0/75 [00:00<?, ?it/s]


(_trainable pid=4026795) Test accuracy 71.34% is not higher than the existing highest accuracy 85.21%. Model not saved.
(_trainable pid=4026795) Epoch 5, Duration: 60.48s, Loss: 0.37, Train Acc: 72.99%, Test Acc: 71.34%
(_trainable pid=4026795) Total connections: 4264225
(_trainable pid=4026795) Edge sliver weights: tensor([0.7311, 0.5473, 0.4787, 0.4682, 0.5653], device='cuda:0',
(_trainable pid=4026795)        grad_fn=<SliceBackward0>)


Train Epoch 6/50:   0%|          | 0/268 [00:00<?, ?it/s]
2024-04-13 06:01:55,688	WARNING tune.py:186 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 


KeyboardInterrupt: 

## Test loaded model on mouse 2

### Load data from mouse 2

In [2]:
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import LabelEncoder
import pickle
import torch
import numpy as np

def get_new_mouse_loader(file_path, batch_size=32):
    """
    Loads data for a new mouse and returns a DataLoader for testing.

    Args:
        - file_path (str): Path to the new mouse data file.
        - batch_size (int): Batch size for the DataLoader.

    Returns:
        - DataLoader: A DataLoader instance containing the new mouse's data.
        - num_nodes (int): Number of nodes in the data.
        - num_classes (int): Number of unique classes in the data.
        - spatial_in_features (int): Number of spatial input features.
    """
    with open(file_path, 'rb') as file:
        data = pickle.load(file)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    if np.issubdtype(data['frame'].dtype, np.object_):
        encoder = LabelEncoder()
        y_encoded = encoder.fit_transform(data['frame'].values)
        y = torch.tensor(y_encoded, dtype=torch.long)
    else:
        y = torch.tensor(data['frame'].values, dtype=torch.float32)

    X = torch.tensor(data.drop(columns=['frame']).values, dtype=torch.float32)

    num_samples = X.size(0)
    num_nodes = X.size(1)
    spatial_in_features = 1  # Assuming each node has only one feature

    # Reshape X to have dimensions (num_samples, num_timesteps, num_nodes, spatial_in_features)
    num_timesteps = 10 #Adjust   
    X = X.view(num_samples, num_timesteps, num_nodes, spatial_in_features)

    num_classes = len(torch.unique(y))

    dataset = TensorDataset(X, y)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    return loader, num_nodes, num_classes, spatial_in_features

In [4]:
from auto_hyperparameter_tuner import *

file_path = "output/spike_trains_with_stimulus_session_732592105_10.pkl"
best_model_path = "saved_models/best_model_Static_STGAT_715093703_85.21.pth"

param_space = {
    "wandb_project": "Predicting Visual Stimulus",
    "wandb_api_key": "7c8d251196fd96d2a93bfb6ffd0005ac030ce42b",
    "num_epochs": 50,
    "lr": 0.000815819,
    "temporal_hidden_dim": 760,
    "spatial_hidden_dim": int(tune.randint(15,20).sample()), 
    "edge_threshold":.25,
    "early_stop_patience": 3,
    "early_stop_delta": 0.01,
    "batch_size": 16,
    "graph_batch_size": 16,
    "temporal_layer_dimension":1,
    "spatial_out_features": int(1),  
    "mouse_number": 715093703,
    "timesteps": 10,
    "Architecture": 'Static_STGAT',
    "num_samples": 1,
    "accumulation_steps": 4,
    "graph_lr": .05,
    "use_auto_corr_matrix": True,
    "file_path":"/proj/STOR/pipiras/Neuropixel/output/spike_trains_with_stimulus_session_715093703_10.pkl"
}

trainer = ModelTrainer(param_space)
new_mouse_loader, num_nodes, num_classes, spatial_in_features = get_new_mouse_loader(file_path)

# Load saved model weights
loaded_weights = torch.load(best_model_path)

trainer.computed_params['num_classes'] = num_classes
trainer.computed_params['spatial_in_features'] = spatial_in_features
trainer.computed_params['num_nodes'] = num_nodes
trainer.computed_params['lstm_input_dim'] = spatial_in_features * num_nodes
trainer.computed_params['temporal_hidden_dim'] = param_space.get('temporal_hidden_dim', 128)
trainer.computed_params['temporal_layer_dimension'] = param_space.get('temporal_layer_dimension', 1)
trainer.computed_params['num_epochs'] = param_space.get('num_epochs', 10)

# Fine-tune model on subset of new mouse data
subset_indices = np.random.choice(len(new_mouse_loader.dataset), size=int(0.2 * len(new_mouse_loader.dataset)), replace=False)
subset_dataset = torch.utils.data.Subset(new_mouse_loader.dataset, subset_indices)
subset_loader = torch.utils.data.DataLoader(subset_dataset, batch_size=param_space["batch_size"], shuffle=True)

# Get X_train from the subset data
X_train = subset_dataset[:][0] 

# Initialize new model with same architecture as loaded model
new_model = trainer.initialize_model(param_space, torch.device("cuda"), num_nodes, X_train)

# Load weights of matching layers from saved model
state_dict = new_model.state_dict()
loaded_state_dict = {k: v for k, v in loaded_weights.items() if k in state_dict and state_dict[k].size() == v.size()}
state_dict.update(loaded_state_dict)
new_model.load_state_dict(state_dict)

def fine_tune_model(model, data_loader, config):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=config["lr"])

    num_epochs = 10  # Num of fine-tuning epochs
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0
        correct_train = 0
        total_train = 0

        for features, labels in data_loader:
            features, labels = features.to(device), labels.to(device)
            labels = labels.squeeze().long()

            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

        train_acc = 100 * correct_train / total_train
        print(f'Epoch {epoch+1}, Loss: {running_loss / len(data_loader):.4f}, Train Acc: {train_acc:.2f}%')

# Fine-tune model
fine_tune_model(new_model, subset_loader, param_space)

# Test fine-tuned model on new mouse data
trainer.test_loaded_model(new_model, new_mouse_loader, num_nodes, num_classes)

Epoch 1, Loss: 3.8014, Train Acc: 20.17%
Epoch 2, Loss: 1.5276, Train Acc: 67.67%
Epoch 3, Loss: 0.4309, Train Acc: 92.16%
Epoch 4, Loss: 0.2660, Train Acc: 94.73%
Epoch 5, Loss: 0.2492, Train Acc: 94.80%
Epoch 6, Loss: 0.2462, Train Acc: 94.71%
Epoch 7, Loss: 0.2448, Train Acc: 94.78%
Epoch 8, Loss: 0.2433, Train Acc: 94.77%
Epoch 9, Loss: 0.2428, Train Acc: 94.77%
Epoch 10, Loss: 0.2420, Train Acc: 94.82%
Testing the loaded model on new mouse data...
Number of nodes: 2717
Number of classes: 119
Input data shape: torch.Size([32, 1, 2717, 1])
Input data shape: torch.Size([32, 1, 2717, 1])
Labels shape: torch.Size([32])
Output shape: torch.Size([32, 119])
Predicted labels: tensor([ 95, 111,  94,  56,  41,  49,  75,  29,  94,  40,  94,  94, 114,  49,
         49,  71,  49,  49,  49,  49,  49,  40, 114,  79,  76,  76,  60,  76,
         76,  76,  76,  51], device='cuda:0')
True labels: tensor([ 5,  5,  5,  5,  5,  5,  5,  5,  5,  5, 49, 49, 49, 49, 49, 49, 49, 49,
        49, 49, 76, 76, 